In [2]:
%load_ext autoreload
%autoreload 2

from typing import Tuple

import jax 
import jax.numpy as jnp 
import numpy 

from craftax.craftax_env import CraftaxSymbolicEnv
from craftax.craftax.craftax_state import EnvParams, EnvState 

from examples.craftax.craftax_plr import ActorCritic 
from examples.craftax.craftax_wrappers import LogWrapper 

from jaxued.environments.maze import Maze, MazeRenderer
from jaxued.wrappers import AutoReplayWrapper

from jaxued.environments.maze import (
    Level,
    make_level_generator,
    make_level_mutator_minimax,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# # Setup the environment
# # maze 
# env = Maze(
#     max_height=13,
#     max_width=13,
#     agent_view_size=5,
#     normalize_obs=True,
# )
# eval_env = env
# sample_random_level = make_level_generator(
#     env.max_height, env.max_width, 13
# )
# env_renderer = MazeRenderer(env, tile_size=8)
# env = AutoReplayWrapper(env)
# env_params = env.default_params 

In [4]:
# craftax 
ENV_CLASS = CraftaxSymbolicEnv
DEFAULT_STATICS = ENV_CLASS.default_static_params()
default_env = ENV_CLASS(DEFAULT_STATICS)
env = LogWrapper(default_env)
env = AutoReplayWrapper(env)
eval_env = env
env_params = env.default_params

from craftax.craftax.world_gen.world_gen import generate_world

def sample_random_level(rng):
    return generate_world(rng, env.default_params, DEFAULT_STATICS)


In [25]:
rng = jax.random.PRNGKey(0)
rng, subrng = jax.random.split(rng, 2)
obs, _ = env.reset_to_level(subrng, sample_random_level(rng), env_params)

In [26]:
obs.shape

(8268,)

In [27]:
obs = jax.tree_util.tree_map(
    lambda x: jnp.repeat(x[None, ...], 32, axis=0)[None, ...],
    obs,
)
obs.shape 

(1, 32, 8268)

In [28]:
network = ActorCritic(env.action_space(env_params).n)

In [29]:
ActorCritic.initialize_carry((32,))[0].shape, ActorCritic.initialize_carry((32,))[1].shape

((32, 512), (32, 512))

In [30]:
network_params = network.init(
        rng, 
        (obs, jnp.zeros((1, 32, ))), 
        ActorCritic.initialize_carry((32,))
    )

In [18]:
network_params['params']['OptimizedLSTMCell_0'].keys()

dict_keys(['hf', 'hg', 'hi', 'ho', 'if', 'ig', 'ii', 'io'])